In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re
import sys, getopt
import csv
import pickle
import copy
import os

from tqdm import tqdm


pd.set_option('display.max_rows', 500)

import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
%matplotlib inline  
import seaborn as sns
sns.set_style("darkgrid")

import umap
from sklearn.decomposition import TruncatedSVD as tsvd

def nearZeroVarDropAuto(df,thresh=0.99):
    vVal=df.var(axis=0).values
    cs=pd.Series(vVal).sort_values(ascending=False).cumsum()
    remove=cs[cs>cs.values[-1]*thresh].index.values
    return df.drop(df.columns[remove],axis=1)

%run SodaKick_download_functions.ipynb


import torch
from torch.utils.data import DataLoader, Dataset
from torch.optim import SGD, Adagrad, Adam, Adagrad
import torch.nn as nn
import torch.nn.functional as F

In [4]:
from ray import tune
from ray.tune import CLIReporter
from ray.tune.schedulers import ASHAScheduler

In [5]:
class EarlyStopping:

    """ Stops the training if loss doesn't improve after a given number of epochs. """

    def __init__(self, patience=3, epsilon=1e-5, keepBest=True, silent=True):

        """
        Args:
            patience (int): Number of epochs without change before stopping the learning (default 3).
            epsilon (float): Minimum change in loss to be considered for early stopping (default 1e-5).
            keepBest (bool): Keep track of the best model (memory consuming).
        """

        self.patience = patience
        self.epsilon = epsilon
        self.counter = 0
        
        self.bestScore = np.inf
     
        self.keepBest = keepBest 
        self.bestModel = None

        self.earlyStop = False
        self.silent = silent

    def __call__(self, loss, model):


        """ Evaluate the loss change between epochs and activates early stop if below epsilon.

        Args:
            loss (float): current loss.
            model (torch model): the current model.
        """

        if loss > self.bestScore - self.epsilon:

            self.counter += 1
            if not self.silent:
                print('EarlyStopping counter: {:d}/{:d}'.format(self.counter,self.patience))

            if self.counter >= self.patience:
                self.earlyStop = True

        else:   

            self.counter = 0
            self.bestScore = loss

            if self.keepBest:
                self.bestModel = copy.deepcopy(model)


In [6]:
class matchesDataset(Dataset):

    """ Extend pytorch Dataset class to include cleaning and training set creation, """
    
    def __init__(self, matches, results):

        self.matches = torch.tensor(matches, dtype=torch.float32)
        self.results = torch.tensor(results, dtype=torch.float32)

    def __len__(self):

        """ Returns the len of the training sample. """
        
        return len(self.matches)
        

    def __getitem__(self, index): 

        """ Returns a word, a context word and a list of negative words for training for a given index. 

        Args:
            index (int): index for the word selection.

        Returns:
            (string, string, list of strings): selected word, context word and a randomly drawn list 
                                               of negative words.

        """
        return self.matches[index], self.results[index]

In [7]:
#https://towardsdatascience.com/quirky-keras-custom-and-asymmetric-loss-functions-for-keras-in-r-a8b5271171fe
def WSE(output, target, a=1.5, b=.5):
    loss = torch.mean(a*torch.minimum(torch.zeros(output.shape[1]),output - target)**2+\
                      b*torch.maximum(torch.zeros(output.shape[1]),output - target)**2)      
    return loss

def WSEl1(output, target, a=1.5, b=.5):
    loss = torch.mean(a*torch.abs(torch.minimum(torch.zeros(output.shape[1]),output - target))+\
                      b*torch.abs(torch.maximum(torch.zeros(output.shape[1]),output - target)))      
    return loss

def WSE2(output, target, a=1.5, b=.5):
    loss = np.mean(a*np.minimum(np.zeros(output.shape[0]),output - target)**2+\
                      b*np.maximum(np.zeros(output.shape[0]),output - target)**2)      
    return loss

def WSEl12(output, target, a=1.5, b=.5):
    loss = np.mean(a*np.abs(np.minimum(np.zeros(output.shape[0]),output - target))+\
                      b*np.abs(np.maximum(np.zeros(output.shape[0]),output - target)))      
    return loss

In [8]:
from sklearn import preprocessing

    
with open(r'/Users/federico comitani/GitHub/sodakick/data/10leagues_inp_2a.pkl', 'rb') as pk:
    inp=pickle.load(pk)
with open(r'/Users/federico comitani/GitHub/sodakick/data/10leagues_out_2a.pkl', 'rb') as pk:
    out=pickle.load(pk)     
    
with open(r'/Users/federico comitani/GitHub/sodakick/data/10leagues_inp_2b.pkl', 'rb') as pk:
    inpb=pickle.load(pk)
with open(r'/Users/federico comitani/GitHub/sodakick/data/10leagues_out_2b.pkl', 'rb') as pk:
    outb=pickle.load(pk)     

inp=np.concatenate([inp,inpb])
out=np.concatenate([out,outb])

def normalize_mins(vec):
    for i in range(vec.shape[0]):
        vec[i][::8]=vec[i][::8]/90

def NormalizeData(data):
    return (data - np.min(data)) / (np.max(data) - np.min(data))

def NormalizeMatrix(data):   
    for i in range(data.shape[1]):
        data[:,i] = NormalizeData(data[:,i])
        
NormalizeMatrix(inp)
np.nan_to_num(inp, copy=False)

normalize_mins(out)

#scaler = preprocessing.StandardScaler().fit(inp)
#inp = scaler.transform(inp)

//miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:22: RuntimeWarning: invalid value encountered in true_divide


In [9]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(
         inp, out, test_size=0.2, random_state=32)

In [10]:
class Net(nn.Module):
    
    def __init__(self, num_layers, num_nodes, scaling_factor, num_nodes_out, final_activation):
        super(Net, self).__init__()
        
        self.fc = []
        self.lr = []
        self.fact = final_activation
        self.nl = num_layers
        power=0
        
        for i in range(self.nl):
            self.fc.append(nn.Linear(int(num_nodes*(scaling_factor**power)), int(num_nodes*(scaling_factor**(power+1)))))
            self.lr.append(nn.LeakyReLU())
            power+=1
        
        self.oupt = nn.Linear(int(num_nodes*(scaling_factor**power)), int(num_nodes_out))
    
    def forward(self, x):
        z = x
        for i in range(self.nl):
            z = self.lr[i](self.fc[i](z))
        z = self.oupt(self.fact(z))
        return z
    
    def clp(self):
        with torch.no_grad():
            for i in range(self.nl):
                self.fc[i].weight.copy_ (self.fc[i].weight.data.clamp(min=0)) 
            self.oupt.weight.copy_ (self.oupt.weight.data.clamp(min=0))

In [11]:
def train(config, model=Net, silent=True, checkpoint_dir=None):
    
    phases = ['train','val']
    
    #x_train, x_test, y_train, y_test = data[0], data[1], data[2], data[3]
    
    training_set = matchesDataset(x_train, y_train)
    trainBatch = torch.utils.data.DataLoader(training_set, batch_size=config['batch_size'], shuffle=config['shuffle'], num_workers=config['num_workers'])

    validation_set = matchesDataset(x_test, y_test)
    valBatch = torch.utils.data.DataLoader(validation_set, batch_size=config['batch_size'], shuffle=config['shuffle'], num_workers=config['num_workers'])

    earlStop = EarlyStopping(patience=config['patience'], keepBest=False)
    
    net = model(config['num_layers'], config['num_nodes'], config['scaling_factor'], 
                config['num_nodes_out'], config['final_activation'])
    
    device = "cpu"
    if torch.cuda.is_available():
        device = "cuda:0"
        if torch.cuda.device_count() > 1:
            net = nn.DataParallel(net)
    net.to(device)
    
    if checkpoint_dir:
        model_state, optimizer_state = torch.load(
            os.path.join(checkpoint_dir, "checkpoint"))
        net.load_state_dict(model_state)
        optimizer.load_state_dict(optimizer_state)

    if config['optim']=='adam':
        optimizer = Adam(net.parameters(), lr=config['lr'])
    elif config['optim']=='adagrad':
        optimizer = Adagrad(net.parameters(), lr=config['lr'])
    else:
        print('optim error')
        return
    
    
    losses=[[],[]]
    mses=[]
    diffs=[]
    exit=False
    
    #for epoch in tqdm(range(epochs), desc='Epoch'):
    for epoch in range(config['epochs']):

        if exit:
            break
            
        for phase in phases:
            if phase == 'train':
                net.train(True) 

                """ Run the training of the model. """    

                losses_batch=[]
                for batchNum, batch in enumerate(trainBatch):

                    x = batch[0]
                    y = batch[1]

                    """ Move batches to GPU if available. """

                    if torch.cuda.is_available():
                        x = x.cuda()
                        y = y.cuda()

                    """ Core of training. """
                    
                    loss = config['loss_f'](net(x), y)

                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()
                    if config['clip']:
                        net.clp()

                    losses_batch.append(loss)

                """ Early stop check. """

                earlStop(loss, net)
                finalepoch = epoch
                
                if earlStop.earlyStop:

                    if not silent:
                        print('Limit loss improvement reached, stopping the training.')
                        
                    exit=True 
                
                #losses[0].append(torch.mean(torch.stack(losses_batch)).detach().cpu().numpy())
            else:
                net.train(False)
                net.eval()
                
                val_loss=0
                val_mse=0
      
                losses_batch=[]
                for batchNum, batch in enumerate(valBatch):

                    x = batch[0]
                    y = batch[1]

                    """ Move batches to GPU if available. """

                    if torch.cuda.is_available():
                        x = x.cuda()
                        y = y.cuda()

                    """ Core of training. """

                    optimizer.zero_grad()
                    output=net(x)
                    target=y
                    loss = config['loss_f'](output, target)

                    #losses_batch.append(loss)
                    val_loss+=loss.detach().numpy()
                    val_mse+=nn.MSELoss()(output, target).detach().numpy()
                    
                #losses[1].append(torch.mean(torch.stack(losses_batch)).detach().cpu().numpy())
                
                with tune.checkpoint_dir(epoch) as checkpoint_dir:
                    path = os.path.join(checkpoint_dir, "checkpoint")
                    torch.save((net.state_dict(), optimizer.state_dict()), path)

                tune.report(loss=(val_loss/batchNum), mse=(val_mse/batchNum))
                #tune.report(loss=torch.mean(torch.stack(losses_batch)))

In [18]:
def revert_output(output,lineup=None):

    reframe=pd.DataFrame(output.reshape(50,8),
                 columns=['minutes','goals','assists','cards_yellow','cards_red','own_goals','goals_against','saves'])
    
    reframe[reframe<0] = 0
    if lineup is not None:
        reframe.index=lineup
        reframe.drop([x for x in reframe.index if x.startswith('dummy')], axis=0, inplace=True)
        
    reframe['minutes']*=90
    byteamframe=pd.concat([reframe.iloc[:25,:].sum(axis=0),reframe.iloc[25:,:].sum(axis=0)], axis=1).T
    
    return reframe, byteamframe[byteamframe.columns[1:]]

In [13]:
print('Baseline WSE: {:.3f}'.format(WSE2(np.array([0]*out[0].shape[0]),out[0])))
print('Baseline WSE l1: {:.3f}'.format(WSEl12(np.array([0]*out[0].shape[0]),out[0])))
print('Baseline MSE: {:.3f}'.format(WSE2(np.array([0]*out[0].shape[0]),out[0], a=1, b=1)))
print('Baseline MSE l1: {:.3f}'.format(WSEl12(np.array([0]*out[0].shape[0]),out[0], a=1, b=1)))

print((out[1]-out[10]).sum())
print((out[50]-out[60]).sum())
print((out[100]-out[110]).sum())

Baseline WSE: 0.230
Baseline WSE l1: 0.139
Baseline MSE: 0.153
Baseline MSE l1: 0.092
-18.0
16.566666666666666
2.0000000000000004


In [20]:
def run_rtune(num_samples=10, max_num_epochs=10, gpus_per_trial=2):
    

    config = {
        "num_layers": tune.choice([1, 2, 3]),
        "num_nodes": inp.shape[1],
        "scaling_factor": tune.sample_from(lambda _: np.random.uniform(low=0.5, high=1.0001)),
        "num_nodes_out": out.shape[1], 
        "final_activation" : torch.tanh,
        "clip": tune.choice([True, False]),
        "batch_size": tune.choice([8, 16, 32, 64]),
        "loss_f": tune.choice([WSE, nn.MSELoss(), nn.L1Loss()]),
        "optim": tune.choice(['adam', 'adagrad']),
        "lr": tune.loguniform(1e-4, 1e-1),
        "shuffle": True,
        "num_workers": 4,
        "patience": 10,
        "epochs": 50
    }
    
    scheduler = ASHAScheduler(
        metric="mse",#"loss",
        mode="min",
        max_t=max_num_epochs,
        grace_period=1,
        reduction_factor=2)
    
    reporter = CLIReporter(
        metric_columns=["loss", "mse", "training_iteration"])
    
    result = tune.run(
        train,
        resources_per_trial={"cpu": 2, "gpu": gpus_per_trial},
        config=config,
        num_samples=num_samples,
        scheduler=scheduler,
        progress_reporter=reporter)

    best_trial = result.get_best_trial("loss", "min", "last")
    print("Best trial config: {}".format(best_trial.config))
    print("Best trial final validation loss: {}".format(
        best_trial.last_result["loss"]))
    print("Best trial final validation mse: {}".format(
        best_trial.last_result["mse"]))

    best_trained_model = Net(best_trial.config['num_layers'], 
                             best_trial.config['num_nodes'], best_trial.config['scaling_factor'], 
                             best_trial.config['num_nodes_out'], best_trial.config['final_activation'])
    device = "cpu"
    if torch.cuda.is_available():
        device = "cuda:0"
        if gpus_per_trial > 1:
            best_trained_model = nn.DataParallel(best_trained_model)
    best_trained_model.to(device)

    best_checkpoint_dir = best_trial.checkpoint.value
    model_state, optimizer_state = torch.load(os.path.join(
        best_checkpoint_dir, "checkpoint"))
    best_trained_model.load_state_dict(model_state)

    return best_trained_model
    #test_acc = test_accuracy(best_trained_model, device)
    #print("Best trial test set accuracy: {}".format(test_acc))

In [21]:
btm = run_rtune(num_samples=50, max_num_epochs=50, gpus_per_trial=0)
#https://docs.ray.io/en/latest/tune/api_docs/suggestion.html#tune-search-alg

2021-05-26 17:18:03,613	WARNING worker.py:1115 -- Warning: The actor ImplicitFunc has size 84676350 when pickled. It will be stored in Redis, which could cause memory issues. This may mean that its definition uses a large array or other object.
2021-05-26 17:18:03,700	WARNING util.py:162 -- The `start_trial` operation took 0.652 s, which may be a performance bottleneck.


== Status ==
Memory usage on this node: 8.7/16.0 GiB
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 32.000: None | Iter 16.000: None | Iter 8.000: None | Iter 4.000: None | Iter 2.000: None | Iter 1.000: None
Resources requested: 2.0/8 CPUs, 0/0 GPUs, 0.0/4.37 GiB heap, 0.0/2.18 GiB objects
Result logdir: /Users/federico comitani/ray_results/train_2021-05-26_17-17-44
Number of trials: 50/50 (49 PENDING, 1 RUNNING)
+-------------------+----------+-------+--------------+--------+-------------------------------+-------------+--------------+---------+------------------+
| Trial name        | status   | loc   |   batch_size | clip   | loss_f                        |          lr |   num_layers | optim   |   scaling_factor |
|-------------------+----------+-------+--------------+--------+-------------------------------+-------------+--------------+---------+------------------|
| train_d0405_00000 | RUNNING  |       |           32 | True   | L1Loss()                      | 0.00667986  |    

2021-05-26 17:18:05,021	WARNING util.py:162 -- The `start_trial` operation took 0.730 s, which may be a performance bottleneck.
2021-05-26 17:18:05,702	WARNING util.py:162 -- The `start_trial` operation took 0.679 s, which may be a performance bottleneck.


Result for train_d0405_00000:
  date: 2021-05-26_17-18-08
  done: false
  experiment_id: 333e5e7a7ae746ee9be3061f7e066609
  hostname: RT5292M-GGB.local
  iterations_since_restore: 1
  loss: 0.20462480577684583
  mse: 0.11665591260506994
  node_ip: 192.168.1.5
  pid: 87911
  should_checkpoint: true
  time_since_restore: 2.398634910583496
  time_this_iter_s: 2.398634910583496
  time_total_s: 2.398634910583496
  timestamp: 1622063888
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: d0405_00000
  
== Status ==
Memory usage on this node: 9.7/16.0 GiB
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 32.000: None | Iter 16.000: None | Iter 8.000: None | Iter 4.000: None | Iter 2.000: None | Iter 1.000: -0.11665591260506994
Resources requested: 8.0/8 CPUs, 0/0 GPUs, 0.0/4.37 GiB heap, 0.0/2.18 GiB objects
Result logdir: /Users/federico comitani/ray_results/train_2021-05-26_17-17-44
Number of trials: 50/50 (46 PENDING, 4 RUNNING)
+-------------------+----------+----------------

2021-05-26 17:18:10,308	WARNING util.py:162 -- The `start_trial` operation took 0.561 s, which may be a performance bottleneck.
2021-05-26 17:18:11,021	WARNING util.py:162 -- The `start_trial` operation took 0.606 s, which may be a performance bottleneck.
2021-05-26 17:18:11,909	WARNING util.py:162 -- The `start_trial` operation took 0.782 s, which may be a performance bottleneck.
2021-05-26 17:18:12,676	WARNING util.py:162 -- The `start_trial` operation took 0.656 s, which may be a performance bottleneck.
2021-05-26 17:18:14,193	WARNING util.py:162 -- The `start_trial` operation took 1.406 s, which may be a performance bottleneck.


...

Result for train_d0405_00047:
  date: 2021-05-26_17-26-29
  done: true
  experiment_id: 6ebea7e0a62441b4abf52e6eb00719f8
  hostname: RT5292M-GGB.local
  iterations_since_restore: 4
  loss: 0.055660580558811915
  mse: 0.055398032792350824
  node_ip: 192.168.1.5
  pid: 91122
  should_checkpoint: true
  time_since_restore: 36.47389078140259
  time_this_iter_s: 7.940635919570923
  time_total_s: 36.47389078140259
  timestamp: 1622064389
  timesteps_since_restore: 0
  training_iteration: 4
  trial_id: d0405_00047
  
Result for train_d0405_00034:
  date: 2021-05-26_17-26-35
  done: false
  experiment_id: 2305e7bb0170491a9a87de349fe49677
  hostname: RT5292M-GGB.local
  iterations_since_restore: 24
  loss: 0.04653151355245534
  mse: 0.04653151355245534
  node_ip: 192.168.1.5
  pid: 90352
  should_checkpoint: true
  time_since_restore: 171.49257016181946
  time_this_iter_s: 3.6069750785827637
  time_total_s: 171.49257016181946
  timestamp: 1622064395
  timesteps_since_restore: 0
  training_itera

2021-05-26 17:26:57,488	INFO logger.py:690 -- Removed the following hyperparameter values when logging to tensorboard: {'loss_f': MSELoss()}


Result for train_d0405_00034:
  date: 2021-05-26_17-26-57
  done: true
  experiment_id: 2305e7bb0170491a9a87de349fe49677
  experiment_tag: 34_batch_size=8,clip=False,loss_f=MSELoss(),lr=0.00017859,num_layers=1,optim=adam,scaling_factor=0.82039
  hostname: RT5292M-GGB.local
  iterations_since_restore: 30
  loss: 0.046365230968769856
  mse: 0.046365230968769856
  node_ip: 192.168.1.5
  pid: 90352
  should_checkpoint: true
  time_since_restore: 193.45936703681946
  time_this_iter_s: 3.6435160636901855
  time_total_s: 193.45936703681946
  timestamp: 1622064417
  timesteps_since_restore: 0
  training_iteration: 30
  trial_id: d0405_00034
  


2021-05-26 17:26:57,912	INFO tune.py:549 -- Total run time: 553.41 seconds (552.79 seconds for the tuning loop).


== Status ==
Memory usage on this node: 9.9/16.0 GiB
Using AsyncHyperBand: num_stopped=41
Bracket: Iter 32.000: None | Iter 16.000: -0.049743526977035595 | Iter 8.000: -0.05087092477640434 | Iter 4.000: -0.05236056363757918 | Iter 2.000: -0.060218450558536196 | Iter 1.000: -0.07403756055761786
Resources requested: 0/8 CPUs, 0/0 GPUs, 0.0/4.37 GiB heap, 0.0/2.18 GiB objects
Result logdir: /Users/federico comitani/ray_results/train_2021-05-26_17-17-44
Number of trials: 50/50 (50 TERMINATED)
+-------------------+------------+-------+--------------+--------+-------------------------------+-------------+--------------+---------+------------------+------------+------------+----------------------+
| Trial name        | status     | loc   |   batch_size | clip   | loss_f                        |          lr |   num_layers | optim   |   scaling_factor |       loss |        mse |   training_iteration |
|-------------------+------------+-------+--------------+--------+----------------------------

In [22]:
pred=btm(torch.Tensor(inp)).detach().cpu().numpy()[3]
cats=['minutes','goals','assists','cards_yellow','cards_red','own_goals']+['goals_against','saves']

reframe, byteamframe = revert_output(pred)
print(byteamframe)
reframe, byteamframe = revert_output(out[3])
print(byteamframe)

      goals   assists  cards_yellow  cards_red  own_goals  goals_against  \
0  0.524458  0.038081      0.173317   0.519007   0.509316       1.307664   
1  0.169901  0.518472      0.627050   0.271117   0.254911       1.080686   

      saves  
0  1.536534  
1  0.516705  
   goals  assists  cards_yellow  cards_red  own_goals  goals_against  saves
0    4.0      4.0           4.0        0.0        0.0            1.0    2.0
1    1.0      1.0           2.0        0.0        0.0            4.0    2.0
